In [2]:
#import the random module
import random

#import pandas for df management
import pandas as pd

# import numpy
import numpy as np

#import matplotlib for plotting funtionality
import matplotlib.pyplot as plt

# Import linear regression from the SciPy stats module.
from scipy.stats import linregress

# import citipy to get closest city from lat long
from citipy import citipy

# import the datetime module from the datetime library
from datetime import datetime

# Import the API key.
from config import weather_api_key

# Import the requests library.
import requests

import os

In [3]:
# create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)

623

In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]

        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
    
# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

    try:
        city_rain = city_weather["rain"]["1h"]
        
    except:
        city_rain = 0.0 
        print("No Rainfall")

    try:
        city_snow = city_weather["snow"]["1h"]
        
    except:
        city_snow = 0.0
        print("No Snowfall")

    city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Description": city_description,
                          "Country": city_country,
                          "Date": city_date,
                          "Rainfall(1h)":city_rain,
                          "Snowfall(1h)":city_snow})

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Snowfall
Processing Record 25 of Set 7 | bambous virieux
No Rainfall
No Snowfall
Processing Record 26 of Set 7 | kefalos
No Rainfall
No Snowfall
Processing Record 27 of Set 7 | sakaiminato
No Rainfall
No Snowfall
Processing Record 28 of Set 7 | jiddah
City not found. Skipping...
No Rainfall
No Snowfall
Processing Record 29 of Set 7 | labuhan
No Rainfall
No Snowfall
Processing Record 30 of Set 7 | hambantota
No Rainfall
No Snowfall
Processing Record 31 of Set 7 | pyay
No Rainfall
No Snowfall
Processing Record 32 of Set 7 | dolbeau
City not found. Skipping...
No Rainfall
No Snowfall
Processing Record 33 of Set 7 | loubomo
No Snowfall
Processing Record 34 of Set 7 | tall kayf
No Rainfall
No Snowfall
Processing Record 35 of Set 7 | phrai bung
City not found. Skipping...
No Rainfall
No Snowfall
Processing Record 36 of Set 7 | araouane
No Rainfall
No Snowfall
Processing Record 37 of Set 7 | gotsu
No Snowfall
Processing Record 38 of Set 7 | meulaboh
No Rainfall
No Snowfall
Processing Record 3

In [6]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date,Rainfall(1h),Snowfall(1h)
0,Fortuna,40.60,-124.16,70.00,72,1,14.99,clear sky,US,2020-07-09 21:13:14,0.00,0.0
1,Grindavik,63.84,-22.43,51.80,71,75,11.41,broken clouds,IS,2020-07-09 21:11:38,0.00,0.0
2,Komsomolskiy,40.43,71.72,77.00,33,100,6.93,overcast clouds,UZ,2020-07-09 21:13:15,0.00,0.0
3,Mataura,-46.19,168.86,42.06,92,100,5.06,light rain,NZ,2020-07-09 21:13:06,0.13,0.0
4,San Ignacio,17.16,-89.07,93.20,56,40,6.93,scattered clouds,BZ,2020-07-09 21:13:15,0.00,0.0


In [7]:
# Create the output file (CSV).
output_data_file = os.path.join("Data","WeatherPy_Database.csv")

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")